In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv('.\AdjustedClosingPrices.csv')
df.head(5)

,Date,A,AA,AAPL,ABC,ABT,ACS,ADBE,ADI,ADM,...,X,XEL,XL,XLNX,XOM,XRAY,XRX,YHOO,YUM,ZION
0,2000-01-03,46.354670,30.172580,3.660058,3.326019,9.639804,0.019,16.274667,32.947692,7.912229,...,25.380447,8.810155,32.639550,35.933610,26.056526,7.151686,19.207609,118.750000,7.452527,43.820247
1,2000-01-04,42.813681,30.312376,3.351477,3.098946,9.364382,0.019,14.909395,31.280900,7.829816,...,24.433043,9.013020,31.170373,35.116937,25.557438,7.151686,18.314232,110.750000,7.302727,41.698321
2,2000-01-05,40.157947,32.059823,3.400523,3.339381,9.347165,0.019,15.204175,31.737555,7.706187,...,24.682360,9.360790,31.448326,34.348304,26.950724,7.275321,19.257241,102.625000,7.340181,41.648973
3,2000-01-06,38.628883,31.640436,3.106247,3.593170,9.674231,0.019,15.328291,30.869910,7.747397,...,24.831950,9.273847,31.686571,31.369849,28.344010,7.284835,18.860185,92.046875,7.277768,42.241139
4,2000-01-07,41.847960,31.547238,3.253385,4.060687,9.777515,0.019,16.072985,31.737555,7.871019,...,25.480174,9.273847,33.791067,34.924779,28.260829,7.265808,19.306873,101.812500,7.115481,42.339833


In [3]:
df['Date'] = pd.to_datetime(df.Date)

In [4]:
df = df.set_index('Date')
df.head(4)

,A,AA,AAPL,ABC,ABT,ACS,ADBE,ADI,ADM,ADP,...,X,XEL,XL,XLNX,XOM,XRAY,XRX,YHOO,YUM,ZION
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-03,46.354670,30.172580,3.660058,3.326019,9.639804,0.019,16.274667,32.947692,7.912229,29.184633,...,25.380447,8.810155,32.639550,35.933610,26.056526,7.151686,19.207609,118.750000,7.452527,43.820247
2000-01-04,42.813681,30.312376,3.351477,3.098946,9.364382,0.019,14.909395,31.280900,7.829816,29.184633,...,24.433043,9.013020,31.170373,35.116937,25.557438,7.151686,18.314232,110.750000,7.302727,41.698321
2000-01-05,40.157947,32.059823,3.400523,3.339381,9.347165,0.019,15.204175,31.737555,7.706187,28.904013,...,24.682360,9.360790,31.448326,34.348304,26.950724,7.275321,19.257241,102.625000,7.340181,41.648973
2000-01-06,38.628883,31.640436,3.106247,3.593170,9.674231,0.019,15.328291,30.869910,7.747397,29.289867,...,24.831950,9.273847,31.686571,31.369849,28.344010,7.284835,18.860185,92.046875,7.277768,42.241139


In [5]:
df.isnull().sum().sum()

184024

In [6]:
df1 = df.replace(0, np.nan)

In [7]:
# fill the missing value
# axis = 0 is add by column 1 means filling by rows
df2 = df1.fillna(method='ffill',axis=0).fillna(method='bfill',axis=0)
df2.isnull().sum().sum()

0

In [8]:
# If the price is 0, change that to 
df_log = np.log(df2)
df_log.isnull().sum().sum()

0

In [9]:
df_r = df_log - df_log.shift()

In [10]:
df_r.isnull().sum().sum()

467

In [11]:
# drop the first row
df_r1 = df_r.drop(df_r.index[0])
df_r1.isnull().sum().sum()

0

In [12]:
today = df_r1.shift(periods=1)[2:]
tomorrow = df_r1[2:]

In [13]:
today_s = today.values
today_s.shape

(4170, 467)

In [14]:
tomorrow_s = tomorrow.values
tomorrow_s.shape

(4170, 467)

In [15]:
# initialize parameters
w = tf.Variable(tf.random_normal([467, 467] ,stddev = 1))
b = tf.Variable(tf.random_normal([1, 467] ,stddev = 1))

In [16]:
x = tf.placeholder(tf.float32, shape=(4170, 467))
y = tf.placeholder(tf.float32, shape=(4170, 467))

In [19]:
loss = tf.reduce_mean(tf.square(y - (tf.matmul(x,w) + b)))

In [20]:
# define optimizer
optimizer = tf.train.AdamOptimizer(learning_rate = 0.01)
train = optimizer.minimize(loss)
init = tf.global_variables_initializer()

In [21]:
sess = tf.Session()
MAXSTEPS = 2000
MOD=1000
#initialize all variables
sess.run(init)

In [22]:
for step in range(MAXSTEPS+1):
    (_,wopt,bopt,loss_opt) = sess.run([train,w ,b,loss],feed_dict={x:today_s, y:tomorrow_s})
    if (step % MOD) == 0:
        print(step, loss_opt)

0 3.9873
1000 0.00517839
2000 0.00487922


In [23]:
tomorrow_s.var()

0.0067300712005412978